In [4]:
!pip install hyperopt

In [6]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix3/workshop_transformacja-3"

/content/drive/My Drive/Colab Notebooks/matrix/matrix3/workshop_transformacja-3


In [7]:
!ls data

dw_signnames  test_road_sign  train_road_sign


In [51]:
import pandas as pd
import numpy as np
import os
import datetime
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin, space_eval

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train = pd.read_pickle('data/train_road_sign')
test = pd.read_pickle('data/test_road_sign')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [11]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((34799, 32, 32, 3), (34799,), (4410, 32, 32, 3), (4410,))

In [0]:
if y_train.ndim == 1:
  y_train = to_categorical(y_train)
if y_test.ndim == 1:
  y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      # validation_data=params_fit.get('validation_data', (X_test, y_test)),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )

  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
  ])

def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [14]:
# get_cnn_v5(input_shape, num_classes).summary()
train_and_predict(get_cnn_v5(input_shape, num_classes))


Epoch 1/5
272/272 [==============================] - 13s 49ms/step - loss: 2.4156 - accuracy: 0.3285 - val_loss: 0.9561 - val_accuracy: 0.7139
Epoch 2/5
272/272 [==============================] - 12s 46ms/step - loss: 0.8584 - accuracy: 0.7264 - val_loss: 0.3495 - val_accuracy: 0.8915
Epoch 3/5
272/272 [==============================] - 12s 46ms/step - loss: 0.4587 - accuracy: 0.8503 - val_loss: 0.1684 - val_accuracy: 0.9457
Epoch 4/5
272/272 [==============================] - 12s 46ms/step - loss: 0.3041 - accuracy: 0.9037 - val_loss: 0.0865 - val_accuracy: 0.9747
Epoch 5/5
272/272 [==============================] - 13s 46ms/step - loss: 0.2486 - accuracy: 0.9243 - val_loss: 0.1120 - val_accuracy: 0.9658


0.9281179138321995

In [0]:
def func_obj(params):
  model = get_cnn_v5(input_shape, num_classes)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=int(params.get('epochs', 5)),
      verbose=int(params.get('verbose', 0)),
  )

  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [16]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.1308 - accuracy: 0.9688

 15/138 [==>...........................]
 - ETA: 0s - loss: 0.1949 - accuracy: 0.9604

 27/138 [====>.........................]
 - ETA: 0s - loss: 0.2489 - accuracy: 0.9479

 39/138 [=======>......................]
 - ETA: 0s - loss: 0.1788 - accuracy: 0.9615

 51/138 [==========>...................]
 - ETA: 0s - loss: 0.1912 - accuracy: 0.9504

 62/138 [============>.................]
 - ETA: 0s - loss: 0.1749 - accuracy: 0.9536


In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj_nn(args):
  model = get_model(args)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(args.get('batch_size', 128)),
      epochs=5, #int(args.get('epochs', 5)),
      verbose=0 #int(args.get('verbose', 0)),
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(args, 'accuracy={}'.format(accuracy))

  return {
      'loss': -accuracy,
      'status': STATUS_OK,
      'eval_time': time.time(),
      'model': model}

In [79]:
space_nn = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0, 1),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0, 1),
}

best = fmin(
    fn=func_obj_nn,
    space=space_nn,
    algo=tpe.suggest,
    max_evals=30,
    trials=Trials()
)
space_eval(space_nn, best)

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.5516514640026787, 'dropout_cnn_block_three': 0.40086781535572935, 'dropout_cnn_block_two': 0.06374518481386171, 'dropout_dense_block_one': 0.22915992378084082, 'dropout_dense_block_two': 0.9776527498500952}
accuracy=0.440362811088562
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.8649225132156719, 'dropout_cnn_block_three': 0.20163848843931675, 'dropout_cnn_block_two': 0.15896160646400814, 'dropout_dense_block_one': 0.2682092312463078, 'dropout_dense_block_two': 0.3562729451420228}
accuracy=0.8514739274978638
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.11909867238413574, 'dropout_cnn_block_three': 0.271603183840886, 'dropout_cnn_block_two': 0.9305380709575187, 'dropout_dense_block_one': 0.6181962654100011, 'dropout_dense_block_two': 0.9033670575104901}
accuracy=0.2825396955013275
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.6705379995047412, 'dropout_cnn_block_three': 0.0074086920609856, 'dropout_cnn_block_two': 0.991958779633

{'batch_size': 140.0,
 'dropout_cnn_block_one': 0.001247412132776765,
 'dropout_cnn_block_three': 0.5903449721834368,
 'dropout_cnn_block_two': 0.4568753123960231,
 'dropout_dense_block_one': 0.3650145555106689,
 'dropout_dense_block_two': 0.13793109924208297}

In [31]:
get_model({
    'dropout_cnn_block_one': 0.5,
    'dropout_cnn_block_two': 0.5,
    'dropout_cnn_block_three': 0.5,
    'dropout_dense_block_one': 0.5,
    'dropout_dense_block_two': 0.5
    }).summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_188 (Conv2D)          (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_189 (Conv2D)          (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_94 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_155 (Dropout)        (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_190 (Conv2D)          (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_191 (Conv2D)          (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_95 (MaxPooling (None, 6, 6, 64)        

In [35]:
space

{'batch_size': <hyperopt.pyll.base.Apply at 0x7fccbc32f358>,
 'dropout_cnn_block_one': <hyperopt.pyll.base.Apply at 0x7fccbc32f400>,
 'dropout_cnn_block_three': <hyperopt.pyll.base.Apply at 0x7fccbc32f208>,
 'dropout_cnn_block_two': <hyperopt.pyll.base.Apply at 0x7fccbc32f7b8>,
 'dropout_dense_block_one': <hyperopt.pyll.base.Apply at 0x7fccbc32f278>,
 'dropout_dense_block_two': <hyperopt.pyll.base.Apply at 0x7fccbc4154e0>}